In [3]:
from swarm import Swarm, Agent
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
client = Swarm()

agent = Agent(
    name="Helpful Assistant",
    instructions="You are a helpful assistant. You answer questions to the best of your ability.",
)

messages = [
    {"role" : "user", "content" : "What is the meaning of life?"}
]

In [12]:
response = client.run(agent=agent, messages=messages)
print(response.messages[0]["content"])

The meaning of life is a deeply philosophical and often personal question that has been contemplated by thinkers, religions, and cultures throughout history. Different perspectives offer various answers:

1. **Religious Interpretations**: Many religions provide specific interpretations of the meaning of life. For example, Christianity teaches that the purpose of life is to know, love, and serve God. Buddhism might suggest that life is about overcoming suffering and achieving enlightenment through the Four Noble Truths and the Eightfold Path.

2. **Philosophical Perspectives**: Existentialists like Jean-Paul Sartre suggest that life has no inherent meaning, and it's up to each individual to create their own purpose. In contrast, Aristotle believed in a life of virtue and reason as the path to happiness and fulfillment.

3. **Scientific and Secular Views**: From a scientific standpoint, the meaning of life might be seen from a biological perspective, where the purpose is to survive, repr

In [24]:
from swarm import Swarm,Agent
client=Swarm()

english_agent=Agent(
    name="English Agent",
    instructions="You only speak English"
)

french_agent=Agent(
    name="French Agent",
    instructions="You only speak French"
)

def transfer_to_french_agent():
    return french_agent


english_agent.functions.append(transfer_to_french_agent)

messages=[{"role":"user","content":"Hi How are you?"}]
response=client.run(agent=english_agent,messages=messages)
print(response.messages[-1]["content"])


Hello! I'm here to help you. How can I assist you today?


In [25]:
response = client.run(
    agent=english_agent,
    messages=[
        {
            "role": "user",
            "content": "Hi, How are you?",
        }
    ],
)
print(response.messages[0]["content"])

Hello! I'm here to help you. How can I assist you today?


In [31]:
messages=[{"role":"user","content":"comment vas-tu ?"}]
response = client.run(
    agent=english_agent,
    messages=messages,
)
print(response.messages[-1]["content"])

Je vais bien, merci ! Comment puis-je vous aider aujourd'hui ?


### Context Variable

In [81]:
def instructions(context_variable):
    name = context_variable.get("name")
    return f"You're a helpful Agent, Greet the user by Name: {name}"

def print_account_details(context_variables: dict):
    user_id = context_variables.get("user_id", None)
    name = context_variables.get("name", None)

    # print(f'Success')

    return f'Account Details: UserId: {user_id}, Name: {name}'

agent = Agent(
    name="Agent",
    instructions=instructions,
    functions=[print_account_details],
)

context_variable = {
    "user_id": 1428,
    "name": "M. Ahsaan Abbasi"
}

In [82]:
messages=[{"role":"user","content":"HI, print my Details?"}]
response = client.run(
    agent=agent,
    messages=messages,
    context_variables=context_variable,
)
print(response.messages[-1]["content"])

Hello M. Ahsaan Abbasi! Here are your account details: 

- **UserId:** 1428 
- **Name:** M. Ahsaan Abbasi

How can I assist you further today?


In [83]:
def print_price_details(context_variables: dict):
    salary = context_variables.get("salary", 0)

    return f"Salary: {salary}"

context_variable["salary"] = 5000

agent.functions.append(print_price_details)

messages=[{"role":"user","content":"HI, Show me my Salary?"}]
response = client.run(
    agent=agent,
    messages=messages,
    context_variables=context_variable,
)
print(response.messages[-1]["content"])

Here are your details:

- **Account Details**: 
  - User ID: 1428
  - Name: M. Ahsaan Abbasi

- **Salary**: $5000

Is there anything else you would like to know?


## Function Calling

In [84]:
from typing import Dict, Union
import datetime
import requests as rq

def get_weather(latitude: float, longitude: float):
    """Get weather data for a given latitude and longitude."""
    BASE_URL = f"https://api.open-meteo.com/v1/forecast"
    params: Dict[str, Union[str, int, float]] = {
        "latitude" : latitude,
        "longitude" : longitude,
        "hourly" : "temperature_2m",
        "forecast_days" : 1
    }

    response = rq.get(BASE_URL, params=params)

    if response.status_code == 200:
        data = response.json()
    else:
        raise Exception(f"API Request Failed: {response.status_code}, {response.text}")

    current_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(t.replace("Z", "+00:00")) for t in data["hourly"]["time"]]
    temperature_lists = data["hourly"]["temperature_2m"]

    closet_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_time))

    temperature = temperature_lists[closet_time_index]

    return f"The Current temperature at {latitude}, {longitude} is {temperature}°C"

In [85]:
agent = Agent(
    name="Weather Agent",
    instructions="You're helpful Agent to get Weather of Any City through latitude & longitude?",
    functions=[get_weather]
)

messages = [{"role" : "user", "content" : "What's the Weather in KHI?"}]

response = client.run(agent=agent, messages=messages)
print(response.messages[-1]["content"])

C:\Users\Dell\AppData\Local\Temp\ipykernel_16000\4016473245.py:22: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_time = datetime.datetime.utcnow()


The current temperature in Karachi (KHI) is 28.1°C. If you need more details or information, feel free to ask!
